# 5강 : q-learning

Supermario, Cartpole, Frozen Lake
Credit by 정원석 ( Jay Wonseok Jung ) 대우교수 - HISS대표, 서울사이버대학교 대우교수

안녕하세요, 학생 여러분! 이번 시간에는 인공지능 에이전트가 게임을 학습하는 방법 중 하나인 Q-learning에 대해 배워보도록 하겠습니다.

우리가 살펴볼 게임은 바로 슈퍼 마리오 브라더스입니다. 마리오는 장애물을 피하고, 적을 물리치면서 목적지까지 도달해야 하는 클래식한 플랫폼 게임이죠. 우리는 마리오 게임 환경을 라이브러리를 통해 구현할 것입니다.

Q-learning은 강화 학습의 한 종류로, 에이전트가 환경과 상호작용하면서 보상을 최대화하는 방향으로 학습하는 알고리즘입니다. 에이전트는 현재 상태에서 가능한 행동들의 가치(Q-value)를 업데이트하면서, 장기적인 관점에서 최선의 행동 정책을 찾아갑니다.

마리오 에이전트는 게임 화면을 보고 현재 상태(시각 정보가 아닌 좌표 등)를 인식한 뒤, Q-learning을 통해 어떤 행동을 취해야 할지 판단하게 됩니다. 점프를 해야 할지, 앞으로 달려야 할지, 적을 밟아야 할지 등을 학습하는 거죠. 에이전트가 목표 지점에 도달하면 보상을 받고, 실패하면 패널티를 받습니다. 이렇게 시행착오를 거듭하면서 에이전트는 점차 게임을 잘 플레이하는 방법을 익히게 될 것입니다.

자, 그럼 이제 마리오 에이전트를 학습시키기 위한 Q-learning 코드를 함께 살펴보도록 할까요? 





# 1.Step 1 설치

필요한 라이브러리 설치
주피터 노트북에서는 `!`를 사용하여 터미널 명령을 실행할 수 있습니다.

In [8]:
!pip3 install --upgrade gym
!pip3 install gym_super_mario_bros nes_py
!pip3 install gym

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.


# Step 2 필요한 라이브러리 임포트


gym: 강화학습 환경을 제공하는 라이브러리
gym_super_mario_bros: Super Mario Bros 게임 환경
JoypadSpace, SIMPLE_MOVEMENT: 조작과 행동을 단순화하기 위한 도구들

In [2]:
from nes_py.wrappers import JoypadSpace
import gym_super_mario_bros
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT
import gym


# 3단계: q-learning 

강화학습에서 에이전트는 환경과 상호작용하면서 보상을 최대화하는 방향으로 학습합니다. 

Q-Learning은 강화학습의 대표적인 알고리즘 중 하나로, Q-Value라는 값을 기반으로 의사결정을 합니다.

Q-Value는 특정 상태(state)에서 특정 행동(action)을 취했을 때 얻을 수 있는 기대 보상의 합을 나타냅니다. 

에이전트는 현재 상태에서 가능한 모든 행동의 Q-Value를 비교하여 가장 높은 Q-Value를 가진 행동을 선택하게 됩니다.

Q-Learning에서는 Q-Table이라는 테이블을 사용하여 각 상태-행동 쌍의 Q-Value를 저장합니다.

에이전트가 환경과 상호작용하면서 얻은 경험을 바탕으로 Q-Table을 업데이트하게 되는데, 이때 사용되는 것이 Q-Learning 업데이트 규칙입니다.


Q(s,a) ← (1 - α) * Q(s,a) + α * [r + γ * max(Q(s',a'))]


Q(s,a): 현재 상태 s에서 행동 a를 취했을 때의 Q-Value

α (learning_rate): 학습률, 새로운 정보를 반영하는 비율

r (reward): 행동 a를 취한 후 받은 보상

γ (discount_factor): 감가율, 미래 보상의 중요도를 조절

max(Q(s',a')): 다음 상태 s'에서 취할 수 있는 행동들 중 가장 높은 Q-Value

이 업데이트 규칙을 통해 에이전트는 장기적인 관점에서 최선의 행동 정책을 학습하게 됩니다.


코드에서는 ε-greedy 방법을 사용하여 행동을 선택합니다. 이는 확률 ε (epsilon)에 따라 무작위 행동을 선택하거나 (탐험, exploration), 현재 Q-Table에서 가장 높은 Q-Value를 가

진 행동을 선택하는 (활용, exploitation) 방식입니다. 이를 통해 에이전트는 새로운 가능성을 탐색하면서도 학습한 지식을 활용할 수 있게 됩니다.

에피소드를 반복하면서 에이전트는 점진적으로 최적의 행동 정책을 학습하게 되고, 이를 통해 슈퍼 마리오 게임을 잘 플레이할 수 있게 됩니다.



In [6]:
import numpy as np
import gym
from gym.spaces import Box
from nes_py.wrappers import JoypadSpace
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT

env = gym.make('SuperMarioBros-v0', apply_api_compatibility=True, render_mode="human")
env = JoypadSpace(env, SIMPLE_MOVEMENT)

# Q-Learning 파라미터
learning_rate = 0.1   # 학습률: Q-Value를 업데이트할 때 사용되는 비율. 0에서 1 사이의 값을 가짐.
discount_factor = 0.99   # 감가율: 미래의 보상을 현재 가치로 환산할 때 사용되는 비율. 0에서 1 사이의 값을 가짐.
epsilon = 0.1   # 탐험을 위한 파라미터: 무작위 행동을 선택할 확률. 0에서 1 사이의 값을 가짐.

n_episodes = 1000   # 총 시도할 에피소드 수

# Q-Table 초기화. 상태와 액션의 수에 맞게 조정할 필요가 있음
# 이 예제에서는 상태와 액션의 수를 간략화하기 위해 임의로 설정함
q_table = np.zeros([env.observation_space.n, env.action_space.n])   # Q-Table: 각 상태-행동 쌍의 Q-Value를 저장하는 테이블

for episode in range(n_episodes):
    state = env.reset()   # 에피소드를 시작할 때마다 환경을 초기화함
    done = False
    total_reward = 0
    
    while not done:
        # ε-greedy 방법으로 행동 선택
        if np.random.uniform(0, 1) < epsilon:
            action = env.action_space.sample()   # 무작위 행동 선택 (탐험)
        else:
            action = np.argmax(q_table[state, :])   # Q-Value가 가장 높은 행동 선택 (활용)
        
        # 환경에서 행동을 취함
        next_state, reward, done, info = env.step(action)
        
        # Q-Value 업데이트 (Q-Learning 업데이트 규칙 적용)
        old_value = q_table[state, action]
        next_max = np.max(q_table[next_state, :])
        new_value = (1 - learning_rate) * old_value + learning_rate * (reward + discount_factor * next_max)
        q_table[state, action] = new_value
        
        total_reward += reward
        state = next_state
    
    if episode % 100 == 0:
        print(f"Episode: {episode}, Total Reward: {total_reward}")

env.close()

AttributeError: 'Box' object has no attribute 'n'

Super Mario Bros 게임 환경에서 발생하는 AttributeError는 env.observation_space와 env.action_space가 연속적이고 복잡한 공간을 나타내기 때문입니다. Super Mario Bros의 경우, observation_space는 게임 화면의 픽셀 데이터를 나타내며, 이는 매우 큰 차원의 데이터입니다. 따라서, 이러한 환경에서는 전통적인 Q-Table 방식을 직접 적용하기 어렵습니다.

대신, Deep Q-Network (DQN)와 같은 방법을 사용하여 복잡한 상태 공간을 처리할 수 있습니다. DQN은 심층 신경망을 사용하여 Q-Value 함수를 근사합니다. 이 접근법은 픽셀 데이터와 같은 고차원 입력을 처리할 수 있으며, 게임과 같은 복잡한 환경에서 효과적입니다.

여기서는 DQN을 구현하는 대신, Q-Learning을 Super Mario Bros에 적용하는 기본적인 아이디어를 간략히 설명하겠습니다. 실제 구현을 위해서는 TensorFlow, PyTorch와 같은 라이브러리를 사용하여 DQN을 구현해야 합니다.



In [ ]:
import gym
import numpy as np
import random
from gym.wrappers import FrameStack, GrayScaleObservation, ResizeObservation
from nes_py.wrappers import JoypadSpace
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT

# 환경을 생성하고 조작 가능한 행동을 SIMPLE_MOVEMENT로 설정합니다.
# 이는 게임을 단순화하여 7개의 기본 동작만 사용할 수 있게 합니다.
env = gym.make('SuperMarioBros-v0', apply_api_compatibility=True, render_mode="human")
env = JoypadSpace(env, SIMPLE_MOVEMENT)

# 환경의 관측을 84x84 크기의 흑백 이미지로 변환하고, 연속적인 4개의 프레임을 하나의 관측으로 취급합니다.
# 이는 에이전트가 시간에 따른 움직임을 인식할 수 있게 해줍니다.
env = ResizeObservation(env, (84, 84))
env = GrayScaleObservation(env, keep_dim=False)
env = FrameStack(env, 4)

# Q-Learning에 사용될 하이퍼파라미터를 설정합니다.
learning_rate = 0.1  # Q-Value 업데이트 시 학습률
discount_factor = 0.99  # 미래 보상에 대한 할인율
epsilon = 0.1  # 탐험을 위한 ε-greedy 전략에서 사용
episodes = 1000  # 총 에피소드 수

# 상태 공간과 행동 공간의 크기를 설정합니다.
# 여기서는 예시로, 상태 공간의 차원과 가능한 행동의 수를 설정합니다.
# 실제로는 이 부분이 실제 게임 상태와 행동을 반영해야 합니다.
num_states = env.observation_space.shape[0]
num_actions = env.action_space.n

# 상태를 인코딩하는 함수입니다. 이 예제에서는 임시 방편으로 상태를 무작위 정수로 변환합니다.
# 실제로는 관측된 환경으로부터 유의미한 상태 정보를 추출하는 과정이 필요합니다.
def encode_state(observation):
    return random.randint(0, num_states - 1)

# Q-Table을 0으로 초기화합니다. 여기서는 각 상태와 행동 쌍에 대한 Q-Value를 저장합니다.
q_table = np.zeros((num_states, num_actions))

# 학습 과정을 시작합니다.
for episode in range(episodes):
    observation = env.reset()  # 환경을 초기화하고 첫 관측을 얻습니다.
    state = encode_state(observation)  # 초기 상태를 얻습니다.
    done = False  # 에피소드 종료 여부를 표시하는 플래그

    while not done:
        if random.uniform(0, 1) < epsilon:
            action = env.action_space.sample()  # ε-greedy 전략에 따라 탐험합니다.
        else:
            action = np.argmax(q_table[state])  # 가장 높은 Q-Value를 가진 행동을 선택합니다.

        # 선택된 행동을 수행하고 새로운 상태와 보상을 얻습니다.
        obs, reward, terminated, truncated, info = env.step(action)
        next_state = encode_state(obs)

        # Q-Value를 업데이트합니다.
        old_value = q_table[state, action]
        next_max = np.max(q_table[next_state])
        
        q_table[state, action] = old_value + learning_rate * (reward + discount_factor * next_max - old_value)

        state = next_state
        done = terminated or truncated

        # 보상과 현재 Q-Value를 출력합니다. 
        print(f"Episode: {episode}, Reward: {reward}, Q-Value: {q_table[state, action]}")



env.close()


2024-03-20 07:53:13.195 Python[88838:5793365] ApplePersistenceIgnoreState: Existing state will not be touched. New state will be written to /var/folders/7b/kknnc53s1w531k3vq6_3br1h0000gn/T/com.apple.python3.savedState
/Users/wonseokjung/Library/Python/3.9/lib/python/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):
2024-03-20 07:53:15.272 Python[88838:5793365] WARNING: Secure coding is automatically enabled for restorable state! However, not on all supported macOS versions of this application. Opt-in to secure coding explicitly by implementing NSApplicationDelegate.applicationSupportsSecureRestorableState:.


Episode: 0, Reward: 0.0, Q-Value: 0.0
Episode: 0, Reward: -1.0, Q-Value: 0.0
Episode: 0, Reward: 0.0, Q-Value: 0.0
Episode: 0, Reward: 0.0, Q-Value: 0.0
Episode: 0, Reward: 0.0, Q-Value: 0.0
Episode: 0, Reward: 0.0, Q-Value: 0.0
Episode: 0, Reward: 0.0, Q-Value: 0.0
Episode: 0, Reward: 0.0, Q-Value: 0.0
Episode: 0, Reward: 0.0, Q-Value: -0.1
Episode: 0, Reward: 0.0, Q-Value: 0.0
Episode: 0, Reward: 0.0, Q-Value: 0.0
Episode: 0, Reward: 0.0, Q-Value: 0.0
Episode: 0, Reward: 0.0, Q-Value: 0.0
Episode: 0, Reward: 0.0, Q-Value: 0.0
Episode: 0, Reward: 0.0, Q-Value: 0.0
Episode: 0, Reward: 0.0, Q-Value: 0.0
Episode: 0, Reward: 0.0, Q-Value: 0.0
Episode: 0, Reward: 0.0, Q-Value: -0.1
Episode: 0, Reward: -1.0, Q-Value: 0.0
Episode: 0, Reward: 0.0, Q-Value: 0.0
Episode: 0, Reward: 0.0, Q-Value: 0.0
Episode: 0, Reward: 0.0, Q-Value: 0.0
Episode: 0, Reward: 0.0, Q-Value: 0.0
Episode: 0, Reward: 0.0, Q-Value: 0.0
Episode: 0, Reward: 0.0, Q-Value: 0.0
Episode: 0, Reward: 0.0, Q-Value: 0.0
Episode:

In [9]:
import gym
import numpy as np
import random

# 환경 생성
env = gym.make('CartPole-v1')

# Q-Learning 하이퍼파라미터
learning_rate = 0.1
discount_factor = 0.99
epsilon = 0.1
episodes = 1000

# 상태 공간을 단순화하기 위한 이산화 파라미터
buckets = (1, 1, 6, 12) # CartPole 상태 변수(위치, 속도, 각도, 각속도)를 이산화하기 위한 구간 수

# 행동 공간의 크기 설정
num_actions = env.action_space.n

# 이산화된 상태 공간의 크기 설정
num_states = np.prod(buckets)

# 상태를 이산화하는 함수
def encode_state(observation):
    upper_bounds = [env.observation_space.high[2], 0.5]
    lower_bounds = [env.observation_space.low[2], -0.5]
    ratios = [(observation[i] + abs(lower_bounds[i])) / (upper_bounds[i] - lower_bounds[i]) for i in range(len(observation))]
    new_obs = [int(round((buckets[i] - 1) * ratios[i])) for i in range(len(observation))]
    new_obs = [min(buckets[i] - 1, max(0, new_obs[i])) for i in range(len(new_obs))]
    return sum([new_obs[i] * np.prod(buckets[:i]) for i in range(len(new_obs))])

# Q-Table 초기화
q_table = np.zeros((num_states, num_actions))

# 학습 과정
for episode in range(episodes):
    observation = env.reset()
    state = encode_state(observation[2:4])  # 위치와 속도를 제외한 각도와 각속도만 사용
    done = False

    while not done:
        if random.uniform(0, 1) < epsilon:
            action = env.action_space.sample()  # 탐험
        else:
            action = np.argmax(q_table[state])  # 활용

        obs, reward, done, info = env.step(action)
        next_state = encode_state(obs[2:4])

        old_value = q_table[state, action]
        next_max = np.max(q_table[next_state])

        # Q-Value 업데이트
        q_table[state, action] = old_value + learning_rate * (reward + discount_factor * next_max - old_value)

        state = next_state

    if episode % 100 == 0:
        print(f"Episode: {episode}")

env.close()


ValueError: too many values to unpack (expected 4)

### 슈퍼마리오보다 더 간단한 환경들

gymnasium 
https://gymnasium.farama.org/index.html

In [10]:
!pip3 install "gymnasium[all]"

Defaulting to user installation because normal site-packages is not writeable
  Using cached box2d-py-2.3.5.tar.gz (374 kB)
  ERROR: Command errored out with exit status 1:
   command: /Library/Developer/CommandLineTools/usr/bin/python3 -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'/private/var/folders/7b/kknnc53s1w531k3vq6_3br1h0000gn/T/pip-install-z3rmr1qg/box2d-py_00ccc27ba01d42268a338b1a63eb43fa/setup.py'"'"'; __file__='"'"'/private/var/folders/7b/kknnc53s1w531k3vq6_3br1h0000gn/T/pip-install-z3rmr1qg/box2d-py_00ccc27ba01d42268a338b1a63eb43fa/setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d /private/var/folders/7b/kknnc53s1w531k3vq6_3br1h0000gn/T/pip-wheel-5r558yc_
       cwd: /private/var/folders/7b/kknnc53s1w531k3vq6_3br1h00

CartPole

In [6]:
import gymnasium as gym
import gymnasium as gym
env=gym.make('CartPole-v1',render_mode="human")

env.reset()
env.render()


2024-03-20 08:12:33.060 Python[89936:5889467] ApplePersistenceIgnoreState: Existing state will not be touched. New state will be written to /var/folders/7b/kknnc53s1w531k3vq6_3br1h0000gn/T/com.apple.python3.savedState


In [14]:
import gym
import numpy as np
import random

# 환경을 생성합니다.
env = gym.make('CartPole-v1', render_mode="human")

# Q-Learning에 사용될 하이퍼파라미터를 설정합니다.
learning_rate = 0.1  # Q-Value 업데이트 시 학습률
discount_factor = 0.99  # 미래 보상에 대한 할인율
epsilon = 0.1  # 탐험을 위한 ε-greedy 전략에서 사용
episodes = 1000  # 총 에피소드 수

# 상태 공간과 행동 공간의 크기를 설정합니다.
num_states = env.observation_space.shape[0]  # 상태 공간의 차원
num_actions = env.action_space.n  # 가능한 행동의 수

# 상태를 인코딩하는 함수입니다.
# 이 예제에서는 CartPole 환경의 상태를 양자화(quantization)하여 이산 상태로 변환합니다.

def encode_state(observation):
    state = 0
    for i, obs in enumerate(observation):
        state += (obs + 2.4) // 0.25 * 4 ** i
    return int(state)

q_table = np.zeros((4096, num_actions))  # 4^4 = 4096


# 학습 과정을 시작합니다.
for episode in range(episodes):
    observation, info = env.reset()  # 환경을 초기화하고 첫 관측을 얻습니다.
    state = encode_state(observation)  # 초기 상태를 얻습니다.
    done = False  # 에피소드 종료 여부를 표시하는 플래그
    total_reward = 0  # 에피소드의 총 보상을 저장할 변수

    while not done:
        if random.uniform(0, 1) < epsilon:
            action = env.action_space.sample()  # ε-greedy 전략에 따라 탐험합니다.
        else:
            action = np.argmax(q_table[state])  # 가장 높은 Q-Value를 가진 행동을 선택합니다.

        # 선택된 행동을 수행하고 새로운 상태와 보상을 얻습니다.
        obs, reward, terminated, truncated, info = env.step(action)
        next_state = encode_state(obs)

        # Q-Value를 업데이트합니다.
        old_value = q_table[state, action]
        next_max = np.max(q_table[next_state])
        q_table[state, action] = old_value + learning_rate * (reward + discount_factor * next_max - old_value)

        state = next_state
        total_reward += reward
        done = terminated or truncated

    # 에피소드마다 총 보상을 출력합니다.
    print(f"Episode: {episode}, Total Reward: {total_reward}")

env.close()

Episode: 0, Total Reward: 10.0
Episode: 1, Total Reward: 11.0
Episode: 2, Total Reward: 9.0
Episode: 3, Total Reward: 10.0
Episode: 4, Total Reward: 9.0
Episode: 5, Total Reward: 9.0
Episode: 6, Total Reward: 9.0
Episode: 7, Total Reward: 9.0
Episode: 8, Total Reward: 8.0
Episode: 9, Total Reward: 8.0
Episode: 10, Total Reward: 11.0
Episode: 11, Total Reward: 10.0
Episode: 12, Total Reward: 11.0
Episode: 13, Total Reward: 10.0
Episode: 14, Total Reward: 10.0
Episode: 15, Total Reward: 20.0
Episode: 16, Total Reward: 12.0
Episode: 17, Total Reward: 8.0
Episode: 18, Total Reward: 10.0
Episode: 19, Total Reward: 11.0
Episode: 20, Total Reward: 10.0
Episode: 21, Total Reward: 12.0
Episode: 22, Total Reward: 13.0
Episode: 23, Total Reward: 11.0
Episode: 24, Total Reward: 10.0
Episode: 25, Total Reward: 10.0
Episode: 26, Total Reward: 9.0
Episode: 27, Total Reward: 9.0
Episode: 28, Total Reward: 10.0
Episode: 29, Total Reward: 10.0
Episode: 30, Total Reward: 9.0
Episode: 31, Total Reward: 11

KeyboardInterrupt: 

### 연습! 
Frozen Lake 환경에서 똑똑한 인공지능 에이전트를 만들어보세요. 
ChatGPT, Claude, Google Gemini 와 같은 LLM을 사용해보시면서 Q-learning을 구현해보고 아래의 코드를 더 효율적이고 효과적으로 바꾸보시는 연습을 해보시기 바랍니다. 


In [13]:
import gym
import numpy as np
import random

# 환경을 생성합니다.
env = gym.make('FrozenLake-v1', render_mode="human")

# Q-Learning에 사용될 하이퍼파라미터를 설정합니다.
learning_rate = 0.8  # Q-Value 업데이트 시 학습률
discount_factor = 0.95  # 미래 보상에 대한 할인율
epsilon = 0.1  # 탐험을 위한 ε-greedy 전략에서 사용
episodes = 2000  # 총 에피소드 수

# 상태 공간과 행동 공간의 크기를 설정합니다.
num_states = env.observation_space.n  # 상태 공간의 크기
num_actions = env.action_space.n  # 가능한 행동의 수

# Q-Table을 0으로 초기화합니다.
q_table = np.zeros((num_states, num_actions))

# 학습 과정을 시작합니다.
for episode in range(episodes):
    state = env.reset()[0]  # 환경을 초기화하고 초기 상태를 얻습니다.
    done = False  # 에피소드 종료 여부를 표시하는 플래그
    
    while not done:
        if random.uniform(0, 1) < epsilon:
            action = env.action_space.sample()  # ε-greedy 전략에 따라 탐험합니다.
        else:
            action = np.argmax(q_table[state])  # 가장 높은 Q-Value를 가진 행동을 선택합니다.
        
        # 선택된 행동을 수행하고 새로운 상태와 보상을 얻습니다.
        next_state, reward, done, _, _ = env.step(action)
        
        # Q-Value를 업데이트합니다.
        old_value = q_table[state, action]
        next_max = np.max(q_table[next_state])
        q_table[state, action] = old_value + learning_rate * (reward + discount_factor * next_max - old_value)
        
        state = next_state
    
    # 100 에피소드마다 현재 에피소드 번호를 출력합니다.
    if episode % 100 == 0:
        print(f"Episode: {episode}")

# 학습이 완료된 후, 최종 정책을 출력합니다.
print("Final Q-Table:")
print(q_table)

Episode: 0
Episode: 100


KeyboardInterrupt: 